In [1]:
#Imports
import cv2
import numpy as np

In [2]:
counter = 0
#Flags to know if the check/invisible and count lines were passed
line_crossed_1 = False     #check/left
line_crossed_2 = False     #count/right
cap = cv2.VideoCapture('video_sidetest.mp4')

#Frames to be compared
ret, frame1 = cap.read()
ret, frame2 = cap.read()

#dimensions = frame1.shape
#First line range  (check/left line)
rangex_1 = range(155,165)
#Second line range (count/right line)
rangex_2 = range(440,485)

width = frame1.shape[1]
height = frame1.shape[0] 
font = cv2.FONT_HERSHEY_SIMPLEX

In [3]:
while(cap.isOpened()):
    #Obtain the absolute difference between orginial and copy frame
    diff = cv2.absdiff(frame1, frame2)
    #Prepare the video frames for processing and find the contours
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    
    #Find the contours of all the objects
    _, contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        #Save the coodinates of all the contours that were identified
        (x, y, w, h) = cv2.boundingRect(contour)
        #Discrimitane any area that is less than the area of a person
        if ((cv2.contourArea(contour)) > 2250 and (h>w)):
            #Draw the green boxes over the body
            cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
            M = cv2.moments(contour)
            #Calculate x,y coordinate of center/blob
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            #Draw blob/center circle
            cv2.circle(frame1,(cx,cy), 5, (0,0,255), -1) 

            #The algorithm to work check if the blob/center, obtained by moments
            #and stored in (cx,cy), of the person/object passed by the different cases,
            #according to that, we (not)count the person
            
            #Cases:
            #Person passed over check line
            if(cx in rangex_1):
                line_crossed_1 = True
                break
            #Person passed over count line
            if((cx in rangex_2) and line_crossed_1 == True):
                line_crossed_2 = True
                break
            #Person walked from right to left
            if(line_crossed_2 == True and line_crossed_1 == False):
                line_crossed_2 = False
                line_crossed_1 = False
                break;
                
            #Check that both lines were crossed
            if(line_crossed_1 == True and line_crossed_2 == True):
                line_crossed_2 = False
                line_crossed_1 = False
                counter += 1
                break
    
    #Entrance/exit to count people
    cv2.rectangle(frame1, (int(width/4), 0), (int(width/4)+5, height), (255, 255, 255), -1)
    #Check line left side
    cv2.rectangle(frame1, (int(width/4)+285, 0), (int(width/4)+325, height), (130, 50, 120), -1)
    #Draw count_txt over frame
    count_txt = 'Count: '+ str(counter)
    cv2.putText(frame1, count_txt ,(10,height-10),font,0.7,(255, 255, 255),1,cv2.LINE_AA)
    
    cv2.imshow("Actual Frame", frame1)
    #Frame refresh
    frame1 = frame2
    ret, frame2 = cap.read()
    
    #ESC for exit
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(3.4.2) /opt/concourse/worker/volumes/live/0851a95b-0a19-4f3a-61a7-502b925fea13/volume/opencv-suite_1535642710601/work/modules/core/src/arithm.cpp:659: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'
